In [2]:
import numpy as np
import pandas as pd
import os
import easygui
import openpyxl
import re

In [3]:
def readexcel(fname):
    df = pd.read_excel(fname,dtype = {'订单号':'str','Offer_ID':'str', 'SKU_ID':'str'})
    return df

def alibabaXiaoshouMingxi(fname):
    df = pd.read_excel(fname,dtype = {'订单号':'str','Offer_ID':'str', 'SKU_ID':'str'})
    col1s = ['订单号','卖家公司名称', '卖家会员名','订单状态', '订单创建时间', '订单付款时间', '收货人姓名', '收货地址', '邮编',
       '联系手机',  '货运公司', '运单号']
    col2s = ['货品总价', '运费（元）', '商家改价（元）',
           '实付款（元）','单价']
    for col in col1s:
        df[col] =  df[col].ffill()
    for col in col2s :
        df[col] = df[col].fillna(0)
    return df

def gethuohao(df0,df1):
    df1 = df1[~df1['订单号'].isin(df0['订单号'].to_list())]
    df = pd.concat([df0,df1])
    return df
    
def getZhuangtai(df):
    # df = df[(df['订单状态'] == '已收货') | (df['订单状态'] == '交易成功')]
    df = df[df['订单状态'] == '交易成功']
    return df

fname0 = r"F:\a00nutstore\008\zw08\电商\阿里巴巴1688\阿里巴巴订单中没有品名(1).xlsx"
fname = r"F:\a00nutstore\008\zw08\电商\阿里巴巴1688\阿里巴巴202240726-20241031.xlsx"
df0 = readexcel(fname0)
df1 = alibabaXiaoshouMingxi(fname)
df = gethuohao(df0,df1)
df = getZhuangtai(df)
df_dingdan = df[['订单号','单品货号','数量','货品总价','运费（元）','商家改价（元）','实付款（元）','单价']]
df_dingdan['jiner'] = df_dingdan['数量']*df_dingdan['单价']
df_dingdan



C:\Users\redda\AppData\Local\Temp\ipykernel_19796\1125658386.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dingdan['jiner'] = df_dingdan['数量']*df_dingdan['单价']


,订单号,单品货号,数量,货品总价,运费（元）,商家改价（元）,实付款（元）,单价,jiner
0,4029296580454477008,EXLA5100Pa,5,10.55,2.8,0.0,13.35,2.11,10.55
1,2278067991350911471,EXLB5100Pa,240,691.20,91.2,0.0,782.40,2.88,691.20
4,2275086218060301182,EXLA6100P,2,4.86,2.8,0.0,7.66,0.57,1.14
5,2275086218060301182,EXBA7100P,3,0.00,0.0,0.0,0.00,1.24,3.72
6,4022437645929203749,XJ404S,900,6109.60,425.0,-1593.6,4941.00,0.61,549.00
...,...,...,...,...,...,...,...,...,...
1635,4024897020184462145,ETCG1630N,1,2.63,2.8,0.0,5.43,0.75,0.75
1636,4024897020184462145,EJCG1630N（分区）,2,0.00,0.0,0.0,0.00,0.94,1.88
1637,2275754989406013772,RJRJ-1680-G01,10,35.90,5.8,0.0,41.70,3.59,35.90
1658,2269186502496746753,EJL3280,10,15.60,4.3,0.0,19.90,1.56,15.60


In [4]:
pivot = df_dingdan.pivot_table(index = '单品货号',values = ['数量','货品总价','运费（元）','商家改价（元）','实付款（元）','jiner'] ,aggfunc = 'sum')
pivot

,jiner,商家改价（元）,实付款（元）,数量,货品总价,运费（元）
单品货号,,,,,,
16K备课本-30P,27.0,0.0,53.7,30,40.6,13.1
16K备课本-50P,13.6,0.0,0.0,10,0.0,0.0
2002,18.9,0.0,0.0,10,0.0,0.0
2003,18.9,0.0,0.0,10,0.0,0.0
2004,18.9,0.0,0.0,10,0.0,0.0
...,...,...,...,...,...,...
XJ402S,488.0,0.0,0.0,800,0.0,0.0
XJ403S,549.0,0.0,0.0,900,0.0,0.0
XJ404S,549.0,-1593.6,4941.0,900,6109.6,425.0


In [5]:
#支付宝收入
fname_shouru = r"F:\a00nutstore\008\zw08\电商\阿里巴巴1688\2088340986968423-20241106-141139155-支付宝卖出交易.csv"
df_shouru = pd.read_csv(fname_shouru,header = 2,encoding = 'gb18030')
df_shouru

,序号,创建时间,支付时间,商品名称,商户订单号,支付宝交易号,对方账户,对方名称,订单金额(元),退款金额(元),...,分公司,银行订单号,交易分类,交易方式,门店,商家实退(元),账期待收款（元）,已收款分期数,付款备注,结算方式
0,1,2024-10-31 23:12:09,,a5笔记本b5空白横线方格记事日记小本批发线圈本简约学生a4厚本子,T50060NP2351395455500418454,2024103122001815371410070274,151******08,**聪,10.40,0.0,...,,,其他,即时到帐,,0.0,,,,
1,2,2024-10-31 21:13:51,2024-11-01 08:44:42,b5活页芯加厚a5方格网格错题活页本a4笔记本活页替换芯批发活页纸,T50060NP2350697260744832573,2024103122001806901456032904,155******73,**花,71.90,0.0,...,,,其他,即时到帐,,0.0,,,,
2,3,2024-10-31 20:29:10,2024-10-31 20:29:17,莱特16k草稿纸按斤卖批发小学生加厚双面演算绘画空白分区草稿本,T50060NP2350301116780724862,2024103122001891761409847907,jjk***@sina.com,*阳,14.10,0.0,...,,1031459010057674,其他,即时到帐,,0.0,,,,
3,4,2024-10-31 20:19:01,2024-10-31 20:19:02,1688增值服务-诚e赊订单红包-订单号:2350804289859268366-莱特作业本...,11200600724103159046899266683,2024103122001130221427561062,sinagf@alibaba-inc.com,阿里巴巴(中国)网络技术有限公司,10.00,0.0,...,,,外部商户,担保交易,,0.0,,,,
4,5,2024-10-31 19:19:15,2024-10-31 19:19:24,莱特b5笔记本记事本a5小本子横线本网格空白本日记本a4笔记本本子,T50060NP2349647284788286071,2024103122001881091443105016,xuz***@yahoo.cn,**华,684.50,0.0,...,,2024103137086546130509090100409,其他,即时到帐,,0.0,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,629,2024-08-28 16:53:20,2024-08-29 10:30:51,莱特作业本16k大本子A5牛皮缝线笔记本练习语文数学英语本子批发,T50060NP4022437645929203749,2024082822001882201449705019,132******50,*鹭,4941.00,896.0,...,,2024082938090251920520090210901,其他,即时到帐,,0.0,,,,
629,630,2024-08-28 10:03:10,2024-08-28 10:06:18,16k大本子A5牛皮缝线本32K作业本笔记本练习语文数学英语本子批发,T50060NP2273049049602822267,2024082822001871461427987104,136******86,**增,116.00,0.0,...,,0828472203654635,其他,即时到帐,,0.0,,,,
630,631,2024-08-24 22:35:04,2024-08-24 22:35:12,作业本胶套日记本b5笔记本子学生用a5小本子练习英语本工作记事本,T50060NP2269186502496746753,2024082422001810421416904066,mor***@163.com,*琼,19.90,0.0,...,,2024082433476804450542090111005,其他,即时到帐,,0.0,,,,
631,632,2024-08-24 17:05:06,2024-08-24 17:28:03,牛皮上翻线圈本B5加厚笔记本子简约考研横线网格空白A6办公记事本,T50060NP2268563451685578186,2024082422001830341423544038,158******52,**群,71.12,0.0,...,,2024082429609438960534090110903,其他,即时到帐,,0.0,,,,


In [6]:
df_shouru1 = df_shouru[['创建时间', '支付时间', '商户订单号','交易状态', '商家实收(元)']]
df_shouru1 = df_shouru1[df_shouru['交易状态'] == '成功']

df_shouru1

,创建时间,支付时间,商户订单号,交易状态,商家实收(元)
4,2024-10-31 19:19:15,2024-10-31 19:19:24,T50060NP2349647284788286071,成功,684.50
22,2024-10-29 20:13:46,2024-10-29 20:13:51,T50060NP4103512671498080632,成功,113.00
24,2024-10-29 12:34:59,2024-10-29 12:35:04,T50060NP2347523473988311663,成功,8.34
27,2024-10-28 22:07:20,2024-10-28 22:07:26,T50060NP2346736370492300393,成功,7.20
47,2024-10-27 14:36:52,2024-10-27 14:37:10,T50060NP4100516282085368020,成功,24.70
...,...,...,...,...,...
627,2024-08-30 10:32:09,2024-08-30 10:32:21,T50060NP2275086218060301182,成功,7.66
628,2024-08-28 16:53:20,2024-08-29 10:30:51,T50060NP4022437645929203749,成功,4045.00
629,2024-08-28 10:03:10,2024-08-28 10:06:18,T50060NP2273049049602822267,成功,116.00
630,2024-08-24 22:35:04,2024-08-24 22:35:12,T50060NP2269186502496746753,成功,19.90


In [7]:
regex1 = re.compile(r'T50060NP(\d+)')  
# string = r'T50060NP2268563451685578186'
# mat = re.search(regex1,string)
# mat
def getDingdanhao(string):
    mat = re.search(regex1,string)
    if mat:
        return mat.group(1)
    else :
        string

# df_shouru1['商户订单号'].map(getDingdanhao)        
        
    
    


In [8]:
df_shouru2 = df_shouru1.assign(订单号 = df_shouru1['商户订单号'].map(getDingdanhao) )
df_shouru3 = df_shouru2[['创建时间','支付时间','订单号','商家实收(元)']]
df_shouru3
   


,创建时间,支付时间,订单号,商家实收(元)
4,2024-10-31 19:19:15,2024-10-31 19:19:24,2349647284788286071,684.50
22,2024-10-29 20:13:46,2024-10-29 20:13:51,4103512671498080632,113.00
24,2024-10-29 12:34:59,2024-10-29 12:35:04,2347523473988311663,8.34
27,2024-10-28 22:07:20,2024-10-28 22:07:26,2346736370492300393,7.20
47,2024-10-27 14:36:52,2024-10-27 14:37:10,4100516282085368020,24.70
...,...,...,...,...
627,2024-08-30 10:32:09,2024-08-30 10:32:21,2275086218060301182,7.66
628,2024-08-28 16:53:20,2024-08-29 10:30:51,4022437645929203749,4045.00
629,2024-08-28 10:03:10,2024-08-28 10:06:18,2273049049602822267,116.00
630,2024-08-24 22:35:04,2024-08-24 22:35:12,2269186502496746753,19.90


In [23]:
result = pd.merge(df_shouru3,df_dingdan,left_on = '订单号',right_on = '订单号',how = 'left')
result['订单号'] = result['订单号'].fillna('8888888888888888888')
result['单品货号'] = result['单品货号'].fillna('A8888')
result = result.assign(jiner = np.where(result['单品货号'] == 'A8888',result['商家实收(元)'],result.jiner))
result

,创建时间,支付时间,订单号,商家实收(元),单品货号,数量,货品总价,运费（元）,商家改价（元）,实付款（元）,单价,jiner
0,2024-10-31 19:19:15,2024-10-31 19:19:24,2349647284788286071,684.50,EFL1660NB,400.0,528.00,156.5,0.0,684.50,0.88,352.00
1,2024-10-31 19:19:15,2024-10-31 19:19:24,2349647284788286071,684.50,EFL1660HB,200.0,0.00,0.0,0.0,0.00,0.88,176.00
2,2024-10-29 20:13:46,2024-10-29 20:13:51,4103512671498080632,113.00,EJCG1630N,50.0,95.00,18.0,0.0,113.00,0.94,47.00
3,2024-10-29 20:13:46,2024-10-29 20:13:51,4103512671498080632,113.00,ETCG1640N,50.0,0.00,0.0,0.0,0.00,0.96,48.00
4,2024-10-29 12:34:59,2024-10-29 12:35:04,2347523473988311663,8.34,EHFB560,4.0,5.84,2.5,0.0,8.34,1.46,5.84
...,...,...,...,...,...,...,...,...,...,...,...,...
836,2024-08-28 16:53:20,2024-08-29 10:30:51,4022437645929203749,4045.00,XJ209S,800.0,0.00,0.0,0.0,0.00,0.61,488.00
837,2024-08-28 16:53:20,2024-08-29 10:30:51,4022437645929203749,4045.00,XJ101S,900.0,0.00,0.0,0.0,0.00,0.61,549.00
838,2024-08-28 10:03:10,2024-08-28 10:06:18,2273049049602822267,116.00,RFDF1660N,100.0,88.00,17.0,11.0,116.00,0.88,88.00
839,2024-08-24 22:35:04,2024-08-24 22:35:12,2269186502496746753,19.90,EJL3280,10.0,15.60,4.3,0.0,19.90,1.56,15.60


In [12]:
result.to_excel('shouru_dingdan分配前.xlsx',index = False)

In [25]:
gp = result.groupby('订单号')
#将订单总金额按各产品的明细销售额进行分配
data = []
for k,v in gp:
    # print(k,v)
    mean = v['商家实收(元)'].mean()
    total = v['jiner'].sum()
    # print('mean',mean)
    # print('total',total)
    v['result_jiner'] = v['商家实收(元)']*(v.jiner)/total
    data.append(v)
result1 = pd.concat(data)
# result1 = result1.reset_index()

result1

,创建时间,支付时间,订单号,商家实收(元),单品货号,数量,货品总价,运费（元）,商家改价（元）,实付款（元）,单价,jiner,result_jiner
840,2024-08-24 17:05:06,2024-08-24 17:28:03,2268563451685578186,71.12,EXBA5100NB,36.0,60.12,11.0,0.0,71.12,1.67,60.12,71.120000
839,2024-08-24 22:35:04,2024-08-24 22:35:12,2269186502496746753,19.90,EJL3280,10.0,15.60,4.3,0.0,19.90,1.56,15.60,19.900000
838,2024-08-28 10:03:10,2024-08-28 10:06:18,2273049049602822267,116.00,RFDF1660N,100.0,88.00,17.0,11.0,116.00,0.88,88.00,116.000000
822,2024-08-30 10:32:09,2024-08-30 10:32:21,2275086218060301182,7.66,EXLA6100P,2.0,4.86,2.8,0.0,7.66,0.57,1.14,1.796790
823,2024-08-30 10:32:09,2024-08-30 10:32:21,2275086218060301182,7.66,EXBA7100P,3.0,0.00,0.0,0.0,0.00,1.24,3.72,5.863210
...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,2024-09-07 13:47:10,2024-09-07 13:47:19,8888888888888888888,48.30,A8888,NaN,NaN,NaN,NaN,NaN,NaN,48.30,5.565233
763,2024-09-03 22:55:26,2024-09-03 22:55:27,8888888888888888888,5.00,A8888,NaN,NaN,NaN,NaN,NaN,NaN,5.00,0.059639
765,2024-09-03 21:16:30,2024-09-03 21:16:31,8888888888888888888,5.00,A8888,NaN,NaN,NaN,NaN,NaN,NaN,5.00,0.059639
778,2024-09-03 02:17:34,2024-09-03 02:17:35,8888888888888888888,0.00,A8888,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.000000


In [26]:
result2 = result1[['创建时间', '支付时间', '订单号', '商家实收(元)', '单品货号', '数量','单价', 'jiner', 'result_jiner']]
result2.columns = ['创建时间', '支付时间', '订单号', '商家实收(元)', '单品货号', '数量','单价', '金额', '最终金额']
result2 = result2.assign(最终金额 = np.where(result2['单品货号'] == 'A8888',result2['商家实收(元)'],result2['最终金额']))
result2

,创建时间,支付时间,订单号,商家实收(元),单品货号,数量,单价,金额,最终金额
840,2024-08-24 17:05:06,2024-08-24 17:28:03,2268563451685578186,71.12,EXBA5100NB,36.0,1.67,60.12,71.12000
839,2024-08-24 22:35:04,2024-08-24 22:35:12,2269186502496746753,19.90,EJL3280,10.0,1.56,15.60,19.90000
838,2024-08-28 10:03:10,2024-08-28 10:06:18,2273049049602822267,116.00,RFDF1660N,100.0,0.88,88.00,116.00000
822,2024-08-30 10:32:09,2024-08-30 10:32:21,2275086218060301182,7.66,EXLA6100P,2.0,0.57,1.14,1.79679
823,2024-08-30 10:32:09,2024-08-30 10:32:21,2275086218060301182,7.66,EXBA7100P,3.0,1.24,3.72,5.86321
...,...,...,...,...,...,...,...,...,...
665,2024-09-07 13:47:10,2024-09-07 13:47:19,8888888888888888888,48.30,A8888,NaN,NaN,48.30,48.30000
763,2024-09-03 22:55:26,2024-09-03 22:55:27,8888888888888888888,5.00,A8888,NaN,NaN,5.00,5.00000
765,2024-09-03 21:16:30,2024-09-03 21:16:31,8888888888888888888,5.00,A8888,NaN,NaN,5.00,5.00000
778,2024-09-03 02:17:34,2024-09-03 02:17:35,8888888888888888888,0.00,A8888,NaN,NaN,0.00,0.00000


In [27]:

# pivot = pivot.reset_index()
total =result2.sum().to_frame().T
result2 = pd.concat([result2,total])
result2.iloc[-1,0] = '小计'
result2.iloc[-1,1] = ''
result2.iloc[-1,2] = ''
result2.iloc[-1,3] = ''
result2.iloc[-1,4] = ''
result2



,创建时间,支付时间,订单号,商家实收(元),单品货号,数量,单价,金额,最终金额
840,2024-08-24 17:05:06,2024-08-24 17:28:03,2268563451685578186,71.12,EXBA5100NB,36.0,1.67,60.12,71.12
839,2024-08-24 22:35:04,2024-08-24 22:35:12,2269186502496746753,19.9,EJL3280,10.0,1.56,15.6,19.9
838,2024-08-28 10:03:10,2024-08-28 10:06:18,2273049049602822267,116.0,RFDF1660N,100.0,0.88,88.0,116.0
822,2024-08-30 10:32:09,2024-08-30 10:32:21,2275086218060301182,7.66,EXLA6100P,2.0,0.57,1.14,1.79679
823,2024-08-30 10:32:09,2024-08-30 10:32:21,2275086218060301182,7.66,EXBA7100P,3.0,1.24,3.72,5.86321
...,...,...,...,...,...,...,...,...,...
763,2024-09-03 22:55:26,2024-09-03 22:55:27,8888888888888888888,5.0,A8888,NaN,NaN,5.0,5.0
765,2024-09-03 21:16:30,2024-09-03 21:16:31,8888888888888888888,5.0,A8888,NaN,NaN,5.0,5.0
778,2024-09-03 02:17:34,2024-09-03 02:17:35,8888888888888888888,0.0,A8888,NaN,NaN,0.0,0.0
817,2024-09-01 17:45:22,2024-09-01 17:45:23,8888888888888888888,5.0,A8888,NaN,NaN,5.0,5.0


In [28]:
result2.to_excel('阿里巴巴销售明细表按品名汇总0726-1031-08-02.xlsx',index = False)

In [ ]:
df.columns

In [ ]:
df_dingdan = df[['订单号','单品货号','数量','货品总价','运费（元）','商家改价（元）','实付款（元）']]
df_dingdan

In [ ]:
import re

string = r'1688买家保障_增值买家保障软件服务费_2342481888793527358'
